In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile5 = pd.read_pickle("BBB_Train_Decile5.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile5.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile5, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))

    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 5')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 5s 2ms/step - loss: 0.4214 - acc: 0.8293 - val_loss: 0.4169 - val_acc: 0.8355
Epoch 39/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4185 - acc: 0.8390 - val_loss: 0.4169 - val_acc: 0.8290
Epoch 40/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4164 - acc: 0.8374 - val_loss: 0.4161 - val_acc: 0.8394
Epoch 41/100
3087/3087 [==============================] - 6s 2ms/step - loss: 0.4198 - acc: 0.8341 - val_loss: 0.4149 - val_acc: 0.8407
Epoch 42/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4112 - acc: 0.8387 - val_loss: 0.4128 - val_acc: 0.8381
Epoch 43/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4142 - acc: 0.8328 - val_loss: 0.4121 - val_acc: 0.8394
Epoch 44/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4140 - acc: 0.8361 - val_loss: 0.4114 - val_acc: 0.8368
Epoch 45/100
3087/3087 [==============================] - 5s 

Epoch 98/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3965 - acc: 0.8371 - val_loss: 0.4011 - val_acc: 0.8394
Epoch 99/100
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3917 - acc: 0.8397 - val_loss: 0.4013 - val_acc: 0.8381
Epoch 100/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4001 - acc: 0.8335 - val_loss: 0.4008 - val_acc: 0.8407
Test accuracy: 0.8406735751295337
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.6067 - acc: 0.6592 - val_loss: 0.5430 - val_acc: 0.7176
Epoch 2/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5113 - acc: 0.7626 - val_loss: 0.4499 - val_acc: 0.8187
Epoch 3/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4658 - acc: 0.8073 - val_loss: 0.4428 - val_acc: 0.8264
Epoch 4/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4434 - acc: 0.8260 - val_l

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3758 - acc: 0.8445 - val_loss: 0.3790 - val_acc: 0.8394
Epoch 57/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3729 - acc: 0.8487 - val_loss: 0.3790 - val_acc: 0.8394
Epoch 58/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3807 - acc: 0.8461 - val_loss: 0.3789 - val_acc: 0.8394
Epoch 59/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3737 - acc: 0.8503 - val_loss: 0.3791 - val_acc: 0.8394
Epoch 60/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3720 - acc: 0.8507 - val_loss: 0.3791 - val_acc: 0.8394
Epoch 61/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3705 - acc: 0.8465 - val_loss: 0.3791 - val_acc: 0.8381
Epoch 62/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3766 - acc: 0.8452 - val_loss: 0.3791 - val_acc: 0.8381
Epoch 63/150
3087/3087 [==============================] - 6s 

3087/3087 [==============================] - 6s 2ms/step - loss: 0.3705 - acc: 0.8452 - val_loss: 0.3796 - val_acc: 0.8420
Epoch 117/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3759 - acc: 0.8419 - val_loss: 0.3797 - val_acc: 0.8433
Epoch 118/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3728 - acc: 0.8432 - val_loss: 0.3797 - val_acc: 0.8407
Epoch 119/150
3087/3087 [==============================] - 6s 2ms/step - loss: 0.3700 - acc: 0.8458 - val_loss: 0.3797 - val_acc: 0.8420
Epoch 120/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3675 - acc: 0.8458 - val_loss: 0.3797 - val_acc: 0.8407
Epoch 121/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3706 - acc: 0.8439 - val_loss: 0.3798 - val_acc: 0.8407
Epoch 122/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.3751 - acc: 0.8422 - val_loss: 0.3796 - val_acc: 0.8407
Epoch 123/150
3087/3087 [==============================

...

Epoch 52/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3754 - acc: 0.8429 - val_loss: 0.3879 - val_acc: 0.8368
Epoch 53/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3742 - acc: 0.8442 - val_loss: 0.3879 - val_acc: 0.8368
Epoch 54/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3728 - acc: 0.8429 - val_loss: 0.3879 - val_acc: 0.8368
Epoch 55/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3665 - acc: 0.8452 - val_loss: 0.3879 - val_acc: 0.8355
Epoch 56/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3717 - acc: 0.8452 - val_loss: 0.3882 - val_acc: 0.8368
Epoch 57/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3698 - acc: 0.8471 - val_loss: 0.3884 - val_acc: 0.8368
Epoch 58/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3772 - acc: 0.8432 - val_loss: 0.3888 - val_acc: 0.8368
Epoch 59/200
3087/3087 [========================

3087/3087 [==============================] - 8s 3ms/step - loss: 0.3713 - acc: 0.8406 - val_loss: 0.3900 - val_acc: 0.8355
Epoch 113/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3762 - acc: 0.8432 - val_loss: 0.3900 - val_acc: 0.8355
Epoch 114/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3710 - acc: 0.8432 - val_loss: 0.3902 - val_acc: 0.8355
Epoch 115/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3731 - acc: 0.8452 - val_loss: 0.3902 - val_acc: 0.8355
Epoch 116/200
3087/3087 [==============================] - 8s 2ms/step - loss: 0.3736 - acc: 0.8400 - val_loss: 0.3901 - val_acc: 0.8355
Epoch 117/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3686 - acc: 0.8442 - val_loss: 0.3904 - val_acc: 0.8355
Epoch 118/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3758 - acc: 0.8474 - val_loss: 0.3904 - val_acc: 0.8355
Epoch 119/200
3087/3087 [==============================

Epoch 172/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3712 - acc: 0.8429 - val_loss: 0.3916 - val_acc: 0.8368
Epoch 173/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3775 - acc: 0.8419 - val_loss: 0.3917 - val_acc: 0.8368
Epoch 174/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3739 - acc: 0.8461 - val_loss: 0.3919 - val_acc: 0.8355
Epoch 175/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3733 - acc: 0.8422 - val_loss: 0.3917 - val_acc: 0.8368
Epoch 176/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3663 - acc: 0.8477 - val_loss: 0.3918 - val_acc: 0.8368
Epoch 177/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3735 - acc: 0.8439 - val_loss: 0.3919 - val_acc: 0.8368
Epoch 178/200
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3749 - acc: 0.8419 - val_loss: 0.3918 - val_acc: 0.8368
Epoch 179/200
3087/3087 [================

3087/3087 [==============================] - 8s 3ms/step - loss: 0.4476 - acc: 0.8192 - val_loss: 0.4413 - val_acc: 0.8238
Epoch 32/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4459 - acc: 0.8238 - val_loss: 0.4412 - val_acc: 0.8303
Epoch 33/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4421 - acc: 0.8212 - val_loss: 0.4400 - val_acc: 0.8251
Epoch 34/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4389 - acc: 0.8286 - val_loss: 0.4397 - val_acc: 0.8303
Epoch 35/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4406 - acc: 0.8247 - val_loss: 0.4389 - val_acc: 0.8329
Epoch 36/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4315 - acc: 0.8264 - val_loss: 0.4383 - val_acc: 0.8329
Epoch 37/100
3087/3087 [==============================] - 8s 3ms/step - loss: 0.4345 - acc: 0.8209 - val_loss: 0.4374 - val_acc: 0.8368
Epoch 38/100
3087/3087 [==============================] - 8s 

3087/3087 [==============================] - 7s 2ms/step - loss: 0.3784 - acc: 0.8416 - val_loss: 0.3967 - val_acc: 0.8316

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 30/50
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3821 - acc: 0.8452 - val_loss: 0.3967 - val_acc: 0.8316
Epoch 31/50
3087/3087 [==============================] - 8s 2ms/step - loss: 0.3768 - acc: 0.8455 - val_loss: 0.3967 - val_acc: 0.8316
Epoch 32/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3760 - acc: 0.8458 - val_loss: 0.3968 - val_acc: 0.8316
Epoch 33/50
3087/3087 [==============================] - 7s 2ms/step - loss: 0.3787 - acc: 0.8452 - val_loss: 0.3969 - val_acc: 0.8303
Epoch 34/50
3087/3087 [==============================] - 8s 3ms/step - loss: 0.3759 - acc: 0.8455 - val_loss: 0.3967 - val_acc: 0.8303
Epoch 35/50
3087/3087 [==============================] - 8s 2ms/step - loss: 0.3815 - acc: 0.8413 - val_loss: 0.3967 - val_acc: 0.8303
Ep

In [ ]:
0.8297